In [1]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict
from scipy.stats import poisson
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import datetime as dt
import random
import time
#time.sleep(2.5*60*60)

%run functions.ipynb
%run underdog_scrape.ipynb
%run pp_scrape.ipynb
#%run parlayplay.ipynb

C:\Users\User\AppData\Local\Temp\ipykernel_12008\3263823425.py:10: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


200



,id,type,attributes.image_url,attributes.league,attributes.league_id,attributes.market,attributes.name,attributes.position,attributes.team,attributes.team_name,relationships.league.data.id,relationships.league.data.type,attributes.lfg_ignored_leagues,attributes.active,attributes.icon,attributes.last_five_games_enabled,attributes.projections_count,attributes.rank,relationships.projection_filters.data
0,67785,new_player,https://static.prizepicks.com/images/players/m...,MLB,2.0,Washington,Patrick Corbin,P,WSH,Nationals,2,league,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,67545,new_player,https://static.prizepicks.com/images/players/m...,MLB,2.0,Houston,Jose Altuve,IF,HOU,Astros,2,league,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63378,new_player,https://s3-us-west-2.amazonaws.com/static.fant...,USFL,191.0,None,Reggie Corbin,RB,Michigan,None,191,league,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63341,new_player,https://i.imgur.com/GhJrHqA.png,USFL,191.0,None,CJ Marable,HB,Birmingham,None,191,league,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39106,new_player,https://s3-us-west-2.amazonaws.com/static.fant...,NFLSZN,163.0,None,Cooper Kupp,WR,LA,None,163,league,NaN,NaN,NaN,NaN,NaN,NaN,NaN


817


C:\Users\User\AppData\Local\Temp\ipykernel_12008\2042246692.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Player'] = df['relationships.new_player.data.id'].map(names_dict)


In [2]:
#scraping data
import urllib3
import json
import gzip

import urllib
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

    

unabated_url = 'https://content.unabated.com/markets/b_playerprops.json?uuid=45370775-9550-48cf-be97-ed8cbe57db0d'
#dog_props_url = 'https://unabated.com/nba/props'
response = opener.open(unabated_url)

http = urllib3.PoolManager()

url = 'https://httpbin.org/post'
myheaders = {'Content-Type': 'application/json'}
mydata = {'username': '***', 'password': '***'}
print(response.status)
decoded_content = gzip.decompress(response.read())
data = json.loads(decoded_content)

200


In [3]:
#team from id dict
teamtemp = pd.json_normalize(data['teams'],max_level=0)
teams = pd.DataFrame()
for i in range(len(teamtemp.T)):
    teams = pd.concat((teams,pd.json_normalize(teamtemp.iloc[0,i])))
teams = teams[['abbreviation','id']]
teamdict = dict(zip(teams.id,teams.abbreviation))

In [4]:
#bookie from id dict
bookies = pd.json_normalize(data['marketSources'])
bookies
id2book = dict(list(zip(bookies.id,bookies.name)))
id2book[13]

'888sports'

In [5]:
#player json normalize
people = pd.json_normalize(data['people'],max_level=0)
ppl = pd.DataFrame()
for i in range(len(people.T)):
    y = pd.json_normalize(people.T.iloc[i])
    ppl = pd.concat((ppl,y))
ppl.sample(3)

,headshotUrl,firstName,lastName,preferredName,jerseyNumber,country,height,weight,position,birthDate,leagueId,createdOn,createdBy,modifiedOn,modifiedBy,id
0,players/mlb/39193.png,Joshua,Bell,Josh,55,USA,76,261,DH,1992-08-14T00:00:00+00:00,5,2022-09-11T03:41:55.737514,-28,2023-06-23T00:00:10.319149,-28,39193
0,players/mlb/39676.png,Miguel,Rojas,Miguel,11,VEN,72,188,IF,1989-02-24T00:00:00+00:00,5,2022-09-11T03:41:57.631654,-28,2023-06-23T00:00:09.179309,-28,39676
0,players/mlb/42029.png,Joseph,Votto,Joey,19,CAN,74,220,IF,1983-09-10T00:00:00+00:00,5,2022-09-11T03:42:16.32079,-28,2023-06-23T00:00:10.421771,-28,42029


In [6]:
i2p = dict(zip(ppl.id,(ppl.firstName+' '+ppl.lastName)))

In [7]:
#step 1 of json normalize for lines
df = pd.json_normalize(data['propsPeopleEvents'],max_level=0)
df2 = pd.json_normalize(df.iloc[0,0])

df2 = pd.DataFrame()
for i in range(len(df.columns)):
    df_temp = pd.json_normalize(df.iloc[0,i],max_level=0)
    df2 = pd.concat((df2,df_temp)).reset_index(drop=True)
df2.playerPosition.unique()

array(['P', 'IF', 'OF', 'C', 'DH', 'G', 'F'], dtype=object)

In [8]:
#replace player id with name
df2.personId = df2.personId.apply(lambda x: i2p[x])

In [9]:
#replacing team id with team name
teams = pd.json_normalize(df2.eventTeams)
teams[['1.id','0.id']]
teams['Team'] =teams['0.id'].apply(lambda x: teamdict[x])
teams['Opp'] =teams['1.id'].apply(lambda x: teamdict[x])
teams = teams[['Team','Opp']]
teams['Opp'][0]

'BAL'

In [10]:
teams

,Team,Opp
0,SEA,BAL
1,SEA,BAL
2,SEA,BAL
3,SEA,BAL
4,SEA,BAL
...,...,...
303,DAL,LAS
304,DAL,LAS
305,DAL,LAS
306,DAL,LAS


In [11]:
#renaming stats
bt2stat = {'bt77': 'TRB',
          'bt73': 'PTS',
          'bt69': '3P',
          'bt78': 'TRB+AST',
          'bt74': 'PTS+AST',
          'bt75': 'PTS+TRB',
           'bt76': 'PTS+TRB+AST',
           'bt70': 'AST',
           'bt84': 'TO',
           'bt82': 'BLK+STL',
           'bt81': 'STL',
           'bt71': 'BLK',
           'bt72': 'Techs',
          'bt83': 'Triple Double',
           'bt77': 'TRB',
          'bt73': 'PTS',
          'bt69': '3P',
          'bt78': 'TRB + AST',
          'bt74': 'PTS+AST',
          'bt75': 'PTS+TRB',
           'bt76': 'PTS+TRB+AST',
           'bt70': 'AST',
           'bt84': 'TO',
           'bt82': 'BLK + STL',
           'bt81': 'STL',
           'bt71': 'BLK',
           'bt72': 'Techs',
          'bt83': 'Triple Double',
          'bt85': 'Total Hits',
          'bt19': 'Total Bases',
          'bt17': 'Pitcher Strikeouts',
          'bt18': 'Home Runs'}

In [12]:
df2['propsMarketSourcesLines'][0].keys()

dict_keys(['si1:ms72:an0', 'si0:ms72:an0'])

In [13]:
#%%timeit

#func to further normalize and aggregate lines for each player
#biggest bottleneck speed wise
def get_odds(row):
    player_name = df2.personId[row]
    guy_base = df2['propsMarketSourcesLines'][row].keys()
    luka = df2['propsMarketSourcesLines'][row]
    test_list = []
    for k in guy_base:
        for k2 in luka[k]:
            new = pd.json_normalize(luka[k][k2])
            new['k'] = k
            book = k.split(':')[1][2:]
            book_name = id2book.get(int(book), 'idk')
            new['book'] = np.where(book_name == 'idk', 'idk', book_name)
            new['side'] = np.where(k.split(':')[0] == 'si0', 'over', 'under')
            new['stat'] = bt2stat.get(k2, k2)
            test_list.append(new)
    test = pd.concat(test_list, axis=0)
    test['player'] = player_name
    test = test[['player','points','price','side','stat','book']].dropna()
    test['Prob'] = test['price'].apply(lambda x: (-x/(100-x)) if x < 0 else 100/(x+100))
    test['Opp'] = teams['Opp'][row]
    test['Team'] = teams['Team'][row]
    return test
luka = get_odds(0)
luka.head()

,player,points,price,side,stat,book,Prob,Opp,Team
0,José Rodríguez,1.5,-135,under,Total Bases,PrizePicks,0.574468,BAL,SEA
0,José Rodríguez,1.5,-135,over,Total Bases,PrizePicks,0.574468,BAL,SEA


In [14]:
#getting lines for every player
odds = pd.DataFrame()
for i in range(len(df2)):
    if i % 20==0:
        print(i,'/',len(df2))
    odds = pd.concat((odds,get_odds(i)))
odds.sample(3)

0 / 308
20 / 308
40 / 308
60 / 308
80 / 308
100 / 308
120 / 308
140 / 308
160 / 308
180 / 308
200 / 308
220 / 308
240 / 308
260 / 308
280 / 308
300 / 308


,player,points,price,side,stat,book,Prob,Opp,Team
0,Gary Sánchez,0.5,-170,over,Total Hits,FanDuel,0.629630,SD,WAS
0,Donovan Solano,1.5,-104,over,Total Bases,TwinSpires,0.509804,DET,MIN
0,Shea Langeliers,0.5,102,under,Total Hits,TwinSpires,0.495050,TOR,OAK


In [15]:
odds.to_csv('Lines/unabated')
odds.sample(2)


,player,points,price,side,stat,book,Prob,Opp,Team
0,Chad Wallach,0.5,-240,over,Total Hits,FanDuel,0.705882,COL,LAA
0,Nicholas Maton,0.5,-140,over,Total Hits,PointsBet,0.583333,DET,MIN


In [16]:
#bookmaker is broken right now
#not sure how to handle Total Bases because it isn't poisson
odds = odds.loc[odds.book !='Bookmaker']
odds = odds.loc[~((odds.stat =='Total Bases') & (odds.points==1.5))]
len(odds)

5688

In [17]:
#getting probabilities to hit every posted line for every player
#probably could use more optimization

columns=['Player','Stat','Line','o_Prob','u_Prob','num_books','means']
players = odds.player.unique()
#players = ['Aaron Gordon']
fullpdf = pd.DataFrame()
for p in players:
    trae = odds.loc[odds.player==p]
    for stat in trae.stat.unique():
        tstat = trae.loc[trae.stat==stat]
        temp = []
        for points in tstat.points.unique():
            tstatline = tstat.loc[tstat.points==points]
            overprob = tstatline.loc[tstatline.side=='over'].Prob.mean()
            underprob = tstatline.loc[tstatline.side=='under'].Prob.mean()
            num_books = len(tstatline)/2
            trueover = (overprob/(overprob+underprob))
            trueunder = (underprob/(overprob+underprob))
            avg = prob2mean(trueover,points,'over')
            temp.append([p,stat,points,round(trueover,2),round(trueunder,2),num_books,avg])
        pdf = pd.DataFrame(temp,columns=columns)
        avg = sum((pdf.means * pdf.num_books))/pdf.num_books.sum()
        pdf['means'] = avg
        pdf.o_Prob=pdf.apply(lambda row: mean2prob(row['means'], row['Line'], 'over'), axis=1)
        pdf.u_Prob=pdf.apply(lambda row: mean2prob(row['means'], row['Line'], 'under'), axis=1)
        fullpdf = pd.concat((fullpdf,pdf))

fullpdf

,Player,Stat,Line,o_Prob,u_Prob,num_books,means
0,Ramón Urías,Home Runs,0.5,0.099692,0.900308,3.0,0.105019
0,Ramón Urías,Total Bases,0.5,0.631577,0.368423,2.0,0.998522
0,Ramón Urías,Total Hits,0.5,0.627536,0.372464,2.5,0.987615
0,Ryan O'Hearn,Total Hits,0.5,0.651464,0.348536,7.5,1.054013
0,Ryan O'Hearn,Home Runs,0.5,0.169467,0.830533,8.0,0.185687
...,...,...,...,...,...,...,...
0,Natasha Howard,TRB,7.5,0.493621,0.506379,4.0,7.625213
0,Natasha Howard,PTS,17.5,0.423084,0.576916,1.0,16.866066
1,Natasha Howard,PTS,16.5,0.519317,0.480683,3.0,16.866066
0,Jordin Canada,PTS,14.5,0.499766,0.500234,4.0,14.665774


In [18]:
fullpdf = fullpdf.reset_index(drop=True)
fullpdf.to_csv('Lines/fullpdf')
fullpdf

,Player,Stat,Line,o_Prob,u_Prob,num_books,means
0,Ramón Urías,Home Runs,0.5,0.099692,0.900308,3.0,0.105019
1,Ramón Urías,Total Bases,0.5,0.631577,0.368423,2.0,0.998522
2,Ramón Urías,Total Hits,0.5,0.627536,0.372464,2.5,0.987615
3,Ryan O'Hearn,Total Hits,0.5,0.651464,0.348536,7.5,1.054013
4,Ryan O'Hearn,Home Runs,0.5,0.169467,0.830533,8.0,0.185687
...,...,...,...,...,...,...,...
750,Natasha Howard,TRB,7.5,0.493621,0.506379,4.0,7.625213
751,Natasha Howard,PTS,17.5,0.423084,0.576916,1.0,16.866066
752,Natasha Howard,PTS,16.5,0.519317,0.480683,3.0,16.866066
753,Jordin Canada,PTS,14.5,0.499766,0.500234,4.0,14.665774


In [19]:
#getting fantasy site lines
#removing weird lines like series, half of quarter
pp_lines = pd.read_csv(f'Lines/full_lines{today.month}_{today.day}',index_col=0)
dog_lines = pd.read_csv(f'Lines/doglines{today.month}_{today.day}',index_col=0)
pp_lines = pp_lines.loc[~pp_lines['League'].isin([250,84,251,231,192,193,227])]
#parlay = pd.read_csv(f'Lines/parlaylines{today.month}_{today.day}',index_col=0)

In [20]:
dog_lines.League.unique()

array(['other', 'MLB', 'NFLSZN', 'WNBA', 'CWS', 'ESPORTS', 'RACING',
       'BASKETBALL'], dtype=object)

In [21]:
pp_lines

,Player,Team,Line,Stat,League,Date
0,Justin Jefferson,2023 MIN,8.5,Rec TDs,163,2023-09-07
1,Zane Smith,Nashville - Rackley Roofing 200,37.5,NASCAR Points,138,2023-06-23
2,Marco Belinelli,Milano,13.5,PTS,254,2023-06-23
3,Patrick Cantlay,RD2,16.0,Pars Or Better,1,2023-06-23
4,Isaiah Briscoe,Enemies,4.5,TRB,38,2023-06-25
...,...,...,...,...,...,...
1356,Denny McCarthy,TPC River Highlands RD 2,68.0,Strokes,1,2023-06-23
1357,Denny McCarthy,TPC River Highlands RD 2,4.5,Birdies Or Better,1,2023-06-23
1358,Ludvig Aberg,TPC River Highlands RD 2,68.5,Strokes,1,2023-06-23
1359,Ludvig Aberg,TPC River Highlands RD 2,4.5,Birdies Or Better,1,2023-06-23


In [22]:
dog_lines

,Line,Stat,Player,League,Team,Date
0,4.50,Aces,Barbora Krejcikova,other,Barbora Krejcikova,NaN
1,2.50,Aces,Linda Fruhvirtova,other,Linda Fruhvirtova,NaN
2,3.50,Double Faults,Barbora Krejcikova,other,Barbora Krejcikova,NaN
3,7.50,Double Faults,Linda Fruhvirtova,other,Linda Fruhvirtova,NaN
4,50.55,Hitter Fantasy Score,Barbora Krejcikova,other,Barbora Krejcikova,NaN
...,...,...,...,...,...,...
1144,1.50,3PM,Jeremy Pargo,BASKETBALL,Bivouac,2023-06-25
1145,8.50,PTS,Tomislav Ivosev,BASKETBALL,Ball Hogs,2023-06-25
1146,7.50,PTS,Amir Johnson,BASKETBALL,Enemies,2023-06-25
1147,12.50,PTS+TRB+AST,Amir Johnson,BASKETBALL,Enemies,2023-06-25


In [23]:
fullpdf

,Player,Stat,Line,o_Prob,u_Prob,num_books,means
0,Ramón Urías,Home Runs,0.5,0.099692,0.900308,3.0,0.105019
1,Ramón Urías,Total Bases,0.5,0.631577,0.368423,2.0,0.998522
2,Ramón Urías,Total Hits,0.5,0.627536,0.372464,2.5,0.987615
3,Ryan O'Hearn,Total Hits,0.5,0.651464,0.348536,7.5,1.054013
4,Ryan O'Hearn,Home Runs,0.5,0.169467,0.830533,8.0,0.185687
...,...,...,...,...,...,...,...
750,Natasha Howard,TRB,7.5,0.493621,0.506379,4.0,7.625213
751,Natasha Howard,PTS,17.5,0.423084,0.576916,1.0,16.866066
752,Natasha Howard,PTS,16.5,0.519317,0.480683,3.0,16.866066
753,Jordin Canada,PTS,14.5,0.499766,0.500234,4.0,14.665774


In [24]:
#finding where prizepicks and underdog are different
fantasy = pp_lines.merge(dog_lines, on=['Player','Stat','Date'],how='inner',suffixes=('_pp','_dog'))
fantasy['difference'] = ((fantasy['Line_pp'] - fantasy['Line_dog']))/fantasy['Line_pp']
fantasy = fantasy.sort_values(by='difference')
fantasy.sample(3)

,Player,Team_pp,Line_pp,Stat,League_pp,Date,Line_dog,League_dog,Team_dog,difference
89,Patrick Corbin,SD,4.0,Pitcher Strikeouts,2,2023-06-23,4.0,MLB,WSH,0.0
10,Shavon Shields,Virtus Bologna,14.5,PTS,254,2023-06-23,14.5,BASKETBALL,Milano,0.0
78,Shane Bieber,MIL,6.0,Pitcher Strikeouts,2,2023-06-23,6.0,MLB,CLE,0.0


In [25]:
temp = fantasy.loc[(fantasy['difference'] != 0.0)]
temp = temp.loc[temp.Stat != 'Hitter Fantasy Score'].sort_values(by='difference')
temp = temp[['Player','Line_pp','Line_dog','Stat', 'difference',  'Date',
       'League_dog', 'Team_dog' ]]
temp.to_csv(f'bets/fantasy_bets_{str(today)}.csv')

In [26]:
temp.loc[temp.Stat != 'Hitter Fantasy Score']

,Player,Line_pp,Line_dog,Stat,difference,Date,League_dog,Team_dog
88,Fernando Tatis Jr.,1.5,2.0,Total Bases,-0.333333,2023-06-23,other,SD
107,EGO,4.5,5.0,MAPS 1-2 Kills,-0.111111,2023-06-23,ESPORTS,Saigon Buffalo
134,hoax,31.0,34.0,MAPS 1-2 Kills,-0.096774,2023-06-23,ESPORTS,The Union
133,happ,32.5,35.5,MAPS 1-2 Kills,-0.092308,2023-06-23,ESPORTS,The Union
16,Elijah Stewart,5.5,6.0,TRB,-0.090909,2023-06-25,BASKETBALL,Trilogy
111,BeanJ,5.5,6.0,MAPS 1-2 Kills,-0.090909,2023-06-23,ESPORTS,Team whales
132,koala,36.5,39.0,MAPS 1-2 Kills,-0.068493,2023-06-23,ESPORTS,The Union
109,Shogun,8.0,8.5,MAPS 1-2 Kills,-0.062500,2023-06-23,ESPORTS,Saigon Buffalo
118,casE,37.0,38.5,MAPS 1-2 Kills,-0.040541,2023-06-23,ESPORTS,HOTU
126,remix,37.0,38.5,MAPS 1-2 Kills,-0.040541,2023-06-23,ESPORTS,ODDIK


In [27]:
temp.loc[temp.League_dog == 'ESPORTS']

,Player,Line_pp,Line_dog,Stat,difference,Date,League_dog,Team_dog
107,EGO,4.5,5.0,MAPS 1-2 Kills,-0.111111,2023-06-23,ESPORTS,Saigon Buffalo
134,hoax,31.0,34.0,MAPS 1-2 Kills,-0.096774,2023-06-23,ESPORTS,The Union
133,happ,32.5,35.5,MAPS 1-2 Kills,-0.092308,2023-06-23,ESPORTS,The Union
111,BeanJ,5.5,6.0,MAPS 1-2 Kills,-0.090909,2023-06-23,ESPORTS,Team whales
132,koala,36.5,39.0,MAPS 1-2 Kills,-0.068493,2023-06-23,ESPORTS,The Union
109,Shogun,8.0,8.5,MAPS 1-2 Kills,-0.062500,2023-06-23,ESPORTS,Saigon Buffalo
118,casE,37.0,38.5,MAPS 1-2 Kills,-0.040541,2023-06-23,ESPORTS,HOTU
126,remix,37.0,38.5,MAPS 1-2 Kills,-0.040541,2023-06-23,ESPORTS,ODDIK
121,darchevile,32.0,33.0,MAPS 1-2 Kills,-0.031250,2023-06-23,ESPORTS,AGO
114,krii,32.5,33.5,MAPS 1-2 Kills,-0.030769,2023-06-23,ESPORTS,The Prodigies


In [28]:
#underdog for now
new = fullpdf.merge(pp_lines, on=['Player','Stat','Line'],how='inner').sort_values(by='o_Prob')
new.to_csv(f'bets/bets_{str(today)}.csv')
new= new.round(3)

new = new.merge(dog_lines, on=['Player','Stat'],how='inner',suffixes=('_pp','_dog'))
new['difference'] = ((new['Line_pp'] - new['Line_dog']))/new['Line_pp']
new

,Player,Stat,Line_pp,o_Prob,u_Prob,num_books,means,Team_pp,League_pp,Date_pp,Line_dog,League_dog,Team_dog,Date_dog,difference
0,James Kaprielian,Pitcher Strikeouts,4.0,0.450,0.550,2.0,3.954,TOR,2,2023-06-23,4.0,MLB,OAK,2023-06-23,0.0
1,Wade Miley,Pitcher Strikeouts,3.0,0.463,0.537,2.0,3.029,CLE,2,2023-06-23,3.0,MLB,MIL,2023-06-23,0.0
2,Patrick Sandoval,Pitcher Strikeouts,5.0,0.465,0.535,2.0,4.996,COL,2,2023-06-23,5.0,MLB,LAA,2023-06-23,0.0
3,Kyle Freeland,Pitcher Strikeouts,3.5,0.472,0.528,7.0,3.542,LAA,2,2023-06-23,3.5,MLB,COL,2023-06-23,0.0
4,Shane Bieber,Pitcher Strikeouts,6.0,0.483,0.517,2.0,6.074,MIL,2,2023-06-23,6.0,MLB,CLE,2023-06-23,0.0
5,Patrick Corbin,Pitcher Strikeouts,4.0,0.487,0.513,1.0,4.105,SD,2,2023-06-23,4.0,MLB,WSH,2023-06-23,0.0
6,Kyle Gibson,Pitcher Strikeouts,5.0,0.488,0.512,2.0,5.103,SEA,2,2023-06-23,5.0,MLB,BAL,2023-06-23,0.0
7,Logan Gilbert,Pitcher Strikeouts,4.5,0.490,0.510,11.0,4.615,BAL,2,2023-06-23,4.5,MLB,SEA,2023-06-23,0.0
8,Kenta Maeda,Pitcher Strikeouts,4.5,0.504,0.496,11.0,4.692,DET,2,2023-06-23,4.5,MLB,MIN,2023-06-23,0.0
9,Brayan Bello,Pitcher Strikeouts,5.0,0.509,0.491,2.0,5.207,CWS,2,2023-06-23,5.0,MLB,BOS,2023-06-23,0.0


In [29]:
new.merge(temp, on=['Player','Stat','Line_pp'],how='inner')

,Player,Stat,Line_pp,o_Prob,u_Prob,num_books,means,Team_pp,League_pp,Date_pp,Line_dog_x,League_dog_x,Team_dog_x,Date_dog,difference_x,Line_dog_y,difference_y,Date,League_dog_y,Team_dog_y


In [30]:
new.columns

Index(['Player', 'Stat', 'Line_pp', 'o_Prob', 'u_Prob', 'num_books', 'means',
       'Team_pp', 'League_pp', 'Date_pp', 'Line_dog', 'League_dog', 'Team_dog',
       'Date_dog', 'difference'],
      dtype='object')

In [31]:
new[['Player', 'Stat', 'Line_pp', 'Line_dog', 'o_Prob', 'u_Prob','Date_pp','difference']].tail(20)

,Player,Stat,Line_pp,Line_dog,o_Prob,u_Prob,Date_pp,difference
0,James Kaprielian,Pitcher Strikeouts,4.0,4.0,0.450,0.550,2023-06-23,0.0
1,Wade Miley,Pitcher Strikeouts,3.0,3.0,0.463,0.537,2023-06-23,0.0
2,Patrick Sandoval,Pitcher Strikeouts,5.0,5.0,0.465,0.535,2023-06-23,0.0
3,Kyle Freeland,Pitcher Strikeouts,3.5,3.5,0.472,0.528,2023-06-23,0.0
4,Shane Bieber,Pitcher Strikeouts,6.0,6.0,0.483,0.517,2023-06-23,0.0
5,Patrick Corbin,Pitcher Strikeouts,4.0,4.0,0.487,0.513,2023-06-23,0.0
6,Kyle Gibson,Pitcher Strikeouts,5.0,5.0,0.488,0.512,2023-06-23,0.0
7,Logan Gilbert,Pitcher Strikeouts,4.5,4.5,0.490,0.510,2023-06-23,0.0
8,Kenta Maeda,Pitcher Strikeouts,4.5,4.5,0.504,0.496,2023-06-23,0.0
9,Brayan Bello,Pitcher Strikeouts,5.0,5.0,0.509,0.491,2023-06-23,0.0


In [32]:
import yagmail
import os

attachment = os.path.abspath(f"bets/bets_{today}.csv")

send = True


if send == True:
    with open(attachment, "rb") as f:
        # send the email with the file object as an attachment

        yag = yagmail.SMTP("sharedwestover@gmail.com", oauth2_file="secrets.json")
        yag.send(subject=f"BETS! {str(today)}",attachments=f)
        
    attachment = os.path.abspath(f"bets/fantasy_bets_{today}.csv")


    with open(attachment, "rb") as f:
        # send the email with the file object as an attachment

        yag = yagmail.SMTP("sharedwestover@gmail.com", oauth2_file="secrets.json")
        yag.send(subject=f"BETS! {str(today)}",attachments=f)